DDIM Noise Comparative Analysis Pipeline

The [P2 weighting (CVPR 2022)](https://arxiv.org/abs/2204.00227) paper proposed an approach to answer the above question, which is their second contribution. The approach consists of the following steps:

1. The input is an image x0.

2. Perturb it to xt using a diffusion process q(xt|x0).

3. strength is a value between 0.0 and 1.0, that controls the amount of noise that is added to the input image. Values that approach 1.0 allow for lots of variations but will also produce images that are not semantically consistent with the input.

4. Reconstruct the image with the learned denoising process pθ(ˆx0|xt).

5. Compare x0 and ˆx0 among various t to show how each step contributes to the sample. The authors used [openai/guided-diffusion](https://github.com/openai/guided-diffusion) model to denoise images in FFHQ dataset. This pipeline extends their second contribution by investigating DDIM on any input image. This script was contributed by [Anh Nguyen](https://github.com/ParagEkbote) and the notebook by [Parag Ekbote](https://github.com/ParagEkbote).

In [3]:
pip install diffusers transformers pillow torch datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from PIL import Image
import numpy as np
from datasets import load_dataset
from diffusers import DiffusionPipeline

# Load the CelebA-HQ dataset
ds = load_dataset("saitsharipov/CelebA-HQ", split="train")

# Get the first image from the dataset
image_data = ds[0]["image"]
image_pil = Image.fromarray(np.array(image_data))  # Convert to PIL image
image_name = "first_image"  # Use a descriptive name for saving files

# Set device
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda")

# Load the diffusion pipeline
pipe = DiffusionPipeline.from_pretrained(
    "google/ddpm-ema-celebahq-256",
    custom_pipeline="ddim_noise_comparative_analysis",
)
pipe = pipe.to(device)

# Perform noise comparative analysis
for strength in np.linspace(0.1, 1, 25):
    denoised_image, latent_timestep = pipe(
        image_pil, strength=strength, return_dict=False
    )
    denoised_image = denoised_image[0]
    denoised_image.save(
        f"noise_comparative_analysis_{image_name}_{latent_timestep}.png"
    )
